## Model Resnet: modules & parameters

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch.nn as nn
import numpy

class channel_selection(nn.Module):
    """
    Select channels from the output of BatchNorm2d layer. It should be put directly after BatchNorm2d layer.
    The output shape of this layer is determined by the number of 1 in `self.indexes`.
    """
    def __init__(self, num_channels):
        """
        Initialize the `indexes` with all one vector with the length same as the number of channels.
        During pruning, the places in `indexes` which correpond to the channels to be pruned will be set to 0.
        """
        super(channel_selection, self).__init__()
        self.indexes = nn.Parameter(torch.ones(num_channels))

    def forward(self, input_tensor):
        """
        Parameter
        ---------
        input_tensor: (N,C,H,W). It should be the output of BatchNorm2d layer.
        """
        selected_index = np.squeeze(np.argwhere(self.indexes.data.cpu().numpy()))
        if selected_index.size == 1:
            selected_index = np.resize(selected_index, (1,)) 
        output = input_tensor[:, selected_index, :, :]
        return output

In [3]:
import math
import torch.nn as nn
import numpy as np

"""
preactivation resnet with bottleneck design.
"""


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, cfg, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.bn1 = nn.BatchNorm2d(inplanes)
        self.select = channel_selection(inplanes)
        self.conv1 = nn.Conv2d(cfg[0], cfg[1], kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(cfg[1])
        self.conv2 = nn.Conv2d(cfg[1], cfg[2], kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(cfg[2])
        self.conv3 = nn.Conv2d(cfg[2], planes * self.expansion, kernel_size=1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        # group1
        out = self.bn1(x)
        out = self.select(out)
        out = self.relu(out)
        out = self.conv1(out)

        # group2
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv2(out)

        # group3
        out = self.bn3(out)
        out = self.relu(out)
        out = self.conv3(out)

        # down sample
        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        return out

    def forward_bn(self, x):
        bn_value = []
        residual = x

        out = self.bn1(x)
        bn_value.append(out.clone())
        out = self.select(out)
        out = self.relu(out)
        out = self.conv1(out)

        out = self.bn2(out)
        bn_value.append(out.clone())
        out = self.relu(out)
        out = self.conv2(out)

        out = self.bn3(out)
        bn_value.append(out.clone())
        out = self.relu(out)
        out = self.conv3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        return out, bn_value

    def mask_bn(self, index, cfg_mask):
        if index == 0:
            self.bn1.weight.data.mul_(cfg_mask)
            self.bn1.bias.data.mul_(cfg_mask)
        elif index == 1:
            self.bn2.weight.data.mul_(cfg_mask)
            self.bn2.bias.data.mul_(cfg_mask)
        elif index == 2:
            self.bn3.weight.data.mul_(cfg_mask)
            self.bn3.bias.data.mul_(cfg_mask)
        else:
            raise ValueError("Index is not including.")


class resnet(nn.Module):
    def __init__(self, depth=164, dataset='cifar10', cfg=None, conv_cfg=None):
        """
        :param depth:
            164 layers => 1 conv2d + 3 layers × 18 blocks (every layer)  × 3 conv2ds (every block)  + 1 avgPool2d
            param n = (depth - 2) // 9:
                n means how many blocks in every layer
                9 = 3 layers × 3 conv2d (every block)
        :param cfg:
            if depth = 164, then len(cfg) = 164
        :param conv_cfg:
            layer block index examples (index starts at 1 & ≤ 18):
                3 indexes / layer: [4, 9, 14] or [6, 12, 18]
                2 indexes / layer: [9, 18]
                1 index / layer: [18]

        number of BatchNorm2d:
            163 = 162 (3 layers × 18 Bottlenecks × 3 BatchNorm2ds) + 1 BatchNorm2d
        """
        super(resnet, self).__init__()
        assert (depth - 2) % 9 == 0, 'depth should be 9n+2'

        n = (depth - 2) // 9  # depth = 164, n = 18
        block = Bottleneck
        self.block_cfg = conv_cfg
        self.inplanes = 16

        # model config
        if cfg is None:
            cfg = [[16, 16, 16], [64, 16, 16] * (n - 1),
                   [64, 32, 32], [128, 32, 32] * (n - 1),
                   [128, 64, 64], [256, 64, 64] * (n - 1), [256]]
            cfg = [item for sub_list in cfg for item in sub_list]

        # model feature
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1, bias=False)
        self.layer1 = self._make_layer(block, 16, n, cfg=cfg[0: 3 * n])
        self.layer2 = self._make_layer(block, 32, n, cfg=cfg[3 * n: 6 * n], stride=2)
        self.layer3 = self._make_layer(block, 64, n, cfg=cfg[6 * n:9 * n], stride=2)
        self.bn = nn.BatchNorm2d(64 * block.expansion)
        self.select = channel_selection(64 * block.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.AvgPool2d(8)

        # model classifier
        if dataset == 'cifar10':
            num_classes = 10
        elif dataset == 'cifar100':
            num_classes = 100
        else:
            raise ValueError('Model `dataset` parameter is Error!')
        self.fc = nn.Linear(cfg[-1], num_classes)

        # model initialize weight
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(0.5)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, cfg, stride=1):
        """
        :param block: Bottleneck item
        :param planes: record the layer's output channel size
                        planes * expansion = output_planes_size,
                        like 16 * 4 = 128 (the first block output)
        :param blocks: number of Bottleneck in layer
        :param cfg:  channel config of all blocks
                        3 cfg items / Bottleneck
        :param stride: default = 1
        """
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False))
        layers = [block(self.inplanes, planes, cfg[0:3], stride, downsample)]
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, cfg[3 * i: 3 * (i + 1)]))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)

        block_value = []
        if self.block_cfg:
            for idx, item in enumerate(self.layer1):
                x = item(x)
                if idx + 1 in self.block_cfg:
                    block_value.append(x)
            for idx, item in enumerate(self.layer2):
                x = item(x)
                if idx + 1 in self.block_cfg:
                    block_value.append(x)
            for idx, item in enumerate(self.layer3):
                x = item(x)
                if idx + 1 in self.block_cfg:
                    block_value.append(x)
        else:
            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)

        x = self.bn(x)
        x = self.select(x)
        x = self.relu(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        y = self.fc(x)

        if len(block_value):
            return y, block_value
        return y


In [4]:
import os

def resume_model(resume_file):
    if not os.path.isfile(resume_file):
        raise ValueError("Resume model file is not found at '{}'".format(resume_file))
    print("=> loading checkpoint '{}'".format(resume_file))
    checkpoint = torch.load(resume_file)
    if 'epoch' in checkpoint:
        start_epoch = checkpoint['epoch']
    else:
        start_epoch = None
        
    if 'best_prec1' in checkpoint:
        best_prec1 = checkpoint['best_prec1']
    else:
        best_prec1 = None
        
    if 'state_dict' in checkpoint:
        state_dict = checkpoint['state_dict']
    else:
        state_dict = None
    
    if 'optimizer' in checkpoint:
        opti_dict = checkpoint['optimizer']
    else:
        opti_dict = None
        
    if 'cfg' in checkpoint:
        cfg = checkpoint['cfg']
        print("-> model cfg is loading...\n cfg: {}".format(list(cfg)))
    else:
        cfg = None
        print("-> not found model cfg...")
    print("=>  epoch {} Prec1: {}".format(start_epoch, best_prec1))
    return state_dict, opti_dict, start_epoch, best_prec1, cfg


data_path = {
    'root': r'D:\Project\Pycharm\network-slimming\logs',
    'root2': r'D:\Project\Gitee\network-slimming\logs',
    'bn': [
        'bn_prune_resnet164_cifar10_percent_0.4',
        'bn_prune_resnet164_cifar10_percent_0.6',
        'bn_prune_resnet164_cifar100_percent_0.4',
        'bn_prune_resnet164_cifar100_percent_0.6',
    ],
    'at': [
         'at_prune_resnet164_cifar10_percent_0.4',
         'at_prune_resnet164_cifar10_percent_0.6',
         'at_prune_resnet164_cifar100_percent_0.4',
         'at_prune_resnet164_cifar100_percent_0.6',
    ],
    'flie': 'model_best.pt',
    'file2': 'model_best.pth.tar',
    'file3': 'pruned.pt',
    'file4': 'pruned.pth.tar'
}

file_path = os.path.join(data_path['root2'], data_path['at'][2], data_path['file3'])
state_dict, opti_dict, start_epoch, best_prec1, cfg = resume_model(file_path)

=> loading checkpoint 'D:\Project\Gitee\network-slimming\logs\at_prune_resnet164_cifar100_percent_0.4\pruned.pt'
-> model cfg is loading...
 cfg: [5, 13, 15, 13, 14, 15, 20, 15, 15, 13, 12, 13, 16, 13, 14, 16, 15, 14, 18, 13, 13, 21, 14, 16, 14, 12, 15, 23, 15, 14, 14, 13, 13, 16, 14, 16, 2, 8, 13, 7, 14, 14, 19, 13, 14, 15, 15, 16, 3, 8, 10, 10, 14, 16, 17, 26, 32, 43, 32, 32, 39, 31, 31, 46, 30, 32, 46, 31, 32, 42, 32, 32, 49, 28, 32, 45, 32, 32, 50, 30, 30, 55, 32, 32, 60, 30, 32, 44, 29, 31, 52, 29, 32, 46, 32, 31, 57, 32, 32, 57, 32, 32, 50, 31, 32, 52, 31, 31, 78, 60, 64, 107, 63, 64, 117, 64, 63, 122, 63, 64, 118, 63, 64, 110, 63, 64, 113, 63, 64, 121, 64, 64, 121, 63, 64, 127, 64, 64, 119, 64, 64, 131, 64, 64, 130, 64, 64, 130, 64, 64, 120, 64, 64, 119, 64, 64, 119, 64, 64, 109, 64, 64, 157]
=>  epoch None Prec1: None


In [5]:
model = resnet(depth=164, dataset='cifar100', cfg=cfg, conv_cfg=None)
model.load_state_dict(state_dict)
model.eval()

resnet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (select): channel_selection()
      (conv1): Conv2d(5, 13, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(13, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(13, 15, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(15, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
    )
    (1): Bottleneck(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (select): channel_select

In [6]:
from torchstat import stat

stat(model, (3, 32, 32))

[MAdd]: channel_selection is not supported!
[Flops]: channel_selection is not supported!
[Memory]: channel_selection is not supported!
[MAdd]: channel_selection is not supported!
[Flops]: channel_selection is not supported!
[Memory]: channel_selection is not supported!
[MAdd]: channel_selection is not supported!
[Flops]: channel_selection is not supported!
[Memory]: channel_selection is not supported!
[MAdd]: channel_selection is not supported!
[Flops]: channel_selection is not supported!
[Memory]: channel_selection is not supported!
[MAdd]: channel_selection is not supported!
[Flops]: channel_selection is not supported!
[Memory]: channel_selection is not supported!
[MAdd]: channel_selection is not supported!
[Flops]: channel_selection is not supported!
[Memory]: channel_selection is not supported!
[MAdd]: channel_selection is not supported!
[Flops]: channel_selection is not supported!
[Memory]: channel_selection is not supported!
[MAdd]: channel_selection is not supported!
[Flops]: ch

In [11]:
import torch
from torchvision import datasets, transforms

kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
test_loader = torch.utils.data.DataLoader(
            datasets.CIFAR100('./data.cifar100', train=True, download=True, transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])),
            batch_size=32, shuffle=True, **kwargs)

Files already downloaded and verified


In [12]:
for k, (data, target) in enumerate(test_loader):
    print(k, data.shape, target)

0 torch.Size([32, 3, 32, 32]) tensor([43, 58,  8, 44, 15, 96, 89, 93, 12, 48, 16, 35, 49,  3, 31, 72, 39, 80,
        20, 70, 22, 27, 91, 10, 88, 12, 82, 29, 54, 13, 89, 96])
1 torch.Size([32, 3, 32, 32]) tensor([79, 88, 72, 66, 91, 64,  3, 30, 47, 13,  7, 17,  5, 13, 66, 69, 37,  7,
        25,  6, 52, 28, 17, 85, 82, 58, 78,  5, 34, 60,  8, 40])
2 torch.Size([32, 3, 32, 32]) tensor([20, 92, 92, 41, 82, 90, 93,  1,  2, 71,  4, 68, 95, 27, 81, 63, 45,  1,
        59, 88, 12, 62, 93, 64,  6, 59, 13,  5, 38, 26, 30, 12])
3 torch.Size([32, 3, 32, 32]) tensor([28, 70, 46, 54, 28, 90, 72, 29, 76, 13,  7, 89, 75, 32, 28, 21, 84, 23,
        49, 27, 69, 18, 29, 60, 94, 40, 51, 49, 80, 17, 27, 76])
4 torch.Size([32, 3, 32, 32]) tensor([14, 57, 28, 69,  0, 24, 36, 36, 90, 78, 54, 21, 74, 46, 67, 91, 82, 54,
        75, 38, 20, 35,  8, 75, 91, 33, 66, 54, 50, 26,  2, 33])
5 torch.Size([32, 3, 32, 32]) tensor([64, 10, 91, 96, 87, 55, 68, 73, 19, 36, 16, 42, 28, 97, 17, 35, 31, 10,
        17, 78,

51 torch.Size([32, 3, 32, 32]) tensor([62, 57, 93, 76, 34, 76,  4, 23, 62, 88, 94, 13, 75, 17, 32, 30, 49, 62,
         1, 27, 48, 87, 55, 64, 42, 10, 17, 62, 79,  4, 87, 63])
52 torch.Size([32, 3, 32, 32]) tensor([49, 46, 61, 62, 73, 74,  2, 53, 58, 56, 40, 95,  2, 25, 84, 11, 52, 66,
        89, 64, 82, 21, 17, 15, 89, 43, 12, 50, 64, 88, 94, 59])
53 torch.Size([32, 3, 32, 32]) tensor([79, 48,  2, 13,  7, 35, 39, 82, 19, 24, 67,  9, 64, 31, 50, 47, 58,  6,
         9, 75,  2, 99, 48, 59, 16, 71, 51, 68, 39, 64,  2, 75])
54 torch.Size([32, 3, 32, 32]) tensor([54, 80, 29, 37,  4, 57, 95, 75, 71,  4,  2,  7, 49, 65, 64, 28, 98, 11,
        74, 89, 37,  6, 57, 67, 18, 29, 38, 27, 92, 71,  8, 85])
55 torch.Size([32, 3, 32, 32]) tensor([44, 12, 65,  3, 60, 44, 48, 51, 74,  8, 85, 68, 74, 79, 24, 68,  5, 55,
        43, 27, 20, 38, 48, 28, 59, 14, 73, 31, 27, 16, 10, 54])
56 torch.Size([32, 3, 32, 32]) tensor([15, 47, 95, 12, 29, 32,  7, 20, 79, 63, 38, 90, 34, 60, 51,  4, 78,  1,
        3

105 torch.Size([32, 3, 32, 32]) tensor([62, 10, 40, 92, 87, 28, 90, 28, 64, 46, 31, 87, 45, 43, 35, 93, 33, 97,
        40, 64, 35, 31, 37, 84, 25, 46,  4, 52, 36, 40, 67, 42])
106 torch.Size([32, 3, 32, 32]) tensor([69, 33, 31,  2, 92, 19, 31, 55, 70, 10, 62, 50, 95, 15, 26, 38, 74,  5,
        58, 17, 91, 81, 89, 12,  6, 19,  5, 88, 60, 46, 61, 88])
107 torch.Size([32, 3, 32, 32]) tensor([22, 12, 33, 89, 92, 24, 80, 30, 71, 43, 84, 62, 76, 83,  0, 21, 37, 30,
        39, 53, 24, 53, 34,  7, 30, 83, 62, 11,  9,  2, 26, 22])
108 torch.Size([32, 3, 32, 32]) tensor([75, 10,  2, 94, 50, 67, 99, 55, 15, 53, 15, 62, 70, 43, 23, 40, 97, 64,
        12, 89, 34, 64,  2, 14, 70, 78, 51, 13, 23, 41, 55, 34])
109 torch.Size([32, 3, 32, 32]) tensor([89, 76, 97, 25, 50, 64, 64, 68,  3, 57, 15, 90, 55, 45, 89, 45, 46, 82,
        16, 75, 83, 70, 66, 46,  6, 24, 75, 33, 48, 15, 25, 37])
110 torch.Size([32, 3, 32, 32]) tensor([45, 76, 86, 41, 60,  1, 44, 43, 61, 61, 47, 34, 84, 86, 40, 65, 23, 13,
   

159 torch.Size([32, 3, 32, 32]) tensor([25,  7, 25, 98, 39, 97, 97, 75, 83, 27, 53, 17, 22, 23, 27, 66, 23, 37,
        38, 32,  0, 99, 47, 97, 10,  9, 73, 62, 81, 76, 87, 71])
160 torch.Size([32, 3, 32, 32]) tensor([55, 49, 47, 74, 54, 82, 57, 61, 84, 35, 83, 27, 28,  5,  8, 87, 40, 57,
        34, 15, 66, 34, 11, 17, 92, 44, 73,  1, 74, 45, 45, 35])
161 torch.Size([32, 3, 32, 32]) tensor([67, 10,  3,  9, 16, 90, 55, 79, 55, 59, 61,  0, 76, 14, 76, 64, 43, 22,
        17, 26, 98, 98, 52, 50, 87, 44, 32, 51, 19, 46, 24, 20])
162 torch.Size([32, 3, 32, 32]) tensor([16, 66, 42, 63, 30, 75, 72, 47, 97,  5, 50, 85, 66, 10, 99, 16, 11, 63,
        55,  0, 19, 98, 68, 95,  3, 48, 16, 34, 42, 88, 74, 43])
163 torch.Size([32, 3, 32, 32]) tensor([20,  9, 87, 33, 44, 18, 71, 65, 33, 50, 84, 97,  8, 33, 65,  4,  0, 51,
        81, 48, 70, 94, 20, 19, 37, 84, 71, 66, 31, 87, 43, 67])
164 torch.Size([32, 3, 32, 32]) tensor([88, 18, 59, 60, 57, 60, 56, 78, 95, 18, 77, 47, 93, 89, 57, 37, 74, 75,
   

        11,  0, 27, 47, 57, 44, 86, 35, 82,  0, 21, 26, 93, 14])
217 torch.Size([32, 3, 32, 32]) tensor([ 2, 31, 77, 37, 76, 51, 43, 23, 40, 28, 25, 53, 53, 98, 62,  4, 88,  7,
        72, 40,  7, 21, 54, 27,  5, 33, 61, 39, 46, 46, 23, 82])
218 torch.Size([32, 3, 32, 32]) tensor([40,  8, 11, 26,  3, 47, 69, 68, 92,  2, 70, 35, 51, 20, 19, 72, 92, 75,
        76, 65,  2, 96, 49, 84, 14, 88, 52, 39, 45,  6,  2, 90])
219 torch.Size([32, 3, 32, 32]) tensor([66, 48, 78, 11, 97, 87, 63, 19, 60, 82,  2, 50, 98,  8, 47, 57, 15, 74,
        43, 49, 22, 69, 71, 12, 51, 71, 47, 58, 98, 31, 91, 68])
220 torch.Size([32, 3, 32, 32]) tensor([71, 64, 59,  5, 94, 85, 21, 93, 17, 38, 38, 36, 41, 98, 20,  6, 96, 97,
        80, 24, 68, 33, 26,  5, 86, 32, 55, 64, 81, 82,  8, 22])
221 torch.Size([32, 3, 32, 32]) tensor([71,  3, 70, 88, 41, 54, 14, 45, 53, 89, 38, 73, 40, 76, 66, 88, 12, 86,
         9,  7, 47, 47, 85,  7, 76, 13, 28, 94, 14, 47, 37, 63])
222 torch.Size([32, 3, 32, 32]) tensor([ 0, 15,  1

...

1297 torch.Size([32, 3, 32, 32]) tensor([27, 18, 46, 47, 68, 87, 75,  5,  0, 81, 81, 10, 71, 78, 99, 32, 28, 76,
        73, 12,  0, 81, 17, 27, 80, 23, 17, 81, 39, 22, 84, 89])
1298 torch.Size([32, 3, 32, 32]) tensor([91, 83, 21, 11, 39, 61, 41, 34, 12, 93, 56, 60, 65, 58, 80, 43,  2, 87,
        72, 36, 67, 89, 69, 34, 57, 54, 77, 49, 49, 41, 39, 33])
1299 torch.Size([32, 3, 32, 32]) tensor([62, 70, 67, 20,  1, 41, 84, 65, 79, 43, 12,  8, 26, 75, 75,  2, 32, 91,
        43,  6, 18, 72,  0,  4, 88, 53, 65, 38,  5, 49, 13, 17])
1300 torch.Size([32, 3, 32, 32]) tensor([31, 75, 70, 65, 57, 32, 79, 23, 83,  3, 49, 40, 66, 55, 29, 45,  5, 46,
        31, 19, 82, 13, 56, 22, 19, 58, 27, 11, 58, 80, 21, 75])
1301 torch.Size([32, 3, 32, 32]) tensor([72, 26, 89, 28, 87, 53, 28, 38, 72, 63,  8, 83, 55, 51,  1, 65, 68,  6,
        45, 62, 39, 92, 97, 31, 42, 66,  3, 16, 98, 79, 10, 25])
1302 torch.Size([32, 3, 32, 32]) tensor([33,  1, 33, 18, 15, 60, 47, 40, 66, 14, 10, 26, 37,  2, 19, 23, 14, 8

1352 torch.Size([32, 3, 32, 32]) tensor([ 7, 66, 47, 37, 30, 80, 51, 35, 65, 83, 71, 80, 53, 47, 98, 67,  7, 10,
        78, 58, 35, 71, 45, 20,  8, 35, 54, 19, 88, 59, 31, 51])
1353 torch.Size([32, 3, 32, 32]) tensor([96, 85, 67, 37, 98, 81,  4, 58, 54, 98, 88, 66, 33, 94, 45, 99, 35, 95,
        74, 17, 42, 39, 22, 95,  2, 37, 95, 20, 97, 56, 24,  9])
1354 torch.Size([32, 3, 32, 32]) tensor([16, 97, 55, 18, 98, 23,  8, 88, 62, 40, 60, 37, 63, 44, 89, 11, 53, 98,
        22, 39, 58, 65, 29, 55, 72, 92,  8, 72, 68, 70, 60, 50])
1355 torch.Size([32, 3, 32, 32]) tensor([73, 90, 11, 14, 32, 21, 22, 39, 85,  7, 37, 98, 46, 27, 16, 59, 50, 58,
        70, 77, 68, 99, 52, 87, 61, 26,  0, 24, 56, 13, 64, 64])
1356 torch.Size([32, 3, 32, 32]) tensor([97,  0, 60, 23, 32, 25,  4, 71,  2, 85, 17, 12, 94, 61, 38, 95, 41, 37,
        82, 95, 58, 36, 42,  8, 63, 30, 66, 24, 66, 58, 87,  0])
1357 torch.Size([32, 3, 32, 32]) tensor([73, 66, 44, 12, 12, 32, 97,  9, 11, 34, 97, 53, 35,  4, 17, 24, 83, 1

1407 torch.Size([32, 3, 32, 32]) tensor([83, 26, 60,  3,  2, 68,  0, 43, 78, 92, 52, 72, 48, 53, 59, 96, 38, 97,
        32, 58, 56, 42, 69, 39, 34, 74, 78, 11,  7, 98, 67, 44])
1408 torch.Size([32, 3, 32, 32]) tensor([77, 85, 54, 74,  2, 69, 42,  5, 61,  3, 99, 27, 61, 11, 48, 97,  0, 56,
        19, 65, 47, 93, 54, 27, 99, 73, 66,  3, 34, 18, 72, 62])
1409 torch.Size([32, 3, 32, 32]) tensor([46, 28, 68, 86,  8, 74, 21, 25, 52, 55, 84, 36, 67, 61, 94, 77, 70, 73,
        74, 31, 81, 21, 55, 28, 18, 78, 51, 66, 26, 24, 67, 66])
1410 torch.Size([32, 3, 32, 32]) tensor([40, 61, 45, 77, 63, 43, 56, 21, 34, 79,  0, 56,  5, 32, 54, 84, 90, 45,
        91, 26, 11, 34, 91,  0, 52, 33, 55, 85, 76, 89, 29, 73])
1411 torch.Size([32, 3, 32, 32]) tensor([36, 30, 75, 66, 22, 43, 17, 81, 17, 82, 69,  3, 56,  0, 85, 56, 39, 41,
        91, 58, 83, 18, 21, 21, 41, 52, 67, 63, 52, 62, 84, 90])
1412 torch.Size([32, 3, 32, 32]) tensor([90, 57, 45, 99, 32, 35, 13, 84, 54, 11, 84, 72,  6, 76, 60, 43,  8, 9

1463 torch.Size([32, 3, 32, 32]) tensor([59, 30, 77, 41, 17, 37, 64, 95,  4,  8, 40, 20, 58, 38, 16, 22, 53, 20,
        36, 28, 65, 75, 92, 76, 73, 45, 48, 84, 73, 58, 91, 24])
1464 torch.Size([32, 3, 32, 32]) tensor([47, 46, 20, 50, 78, 89, 94, 20,  0, 91, 39, 86, 75,  7, 17, 59, 16, 40,
         8, 27,  5, 21, 53, 49, 71, 65,  2, 50, 35, 90, 59, 87])
1465 torch.Size([32, 3, 32, 32]) tensor([66, 32, 94, 47, 80,  2, 79, 92, 80, 92, 35, 37, 17, 81, 76, 49, 89, 17,
        54, 11, 81, 85, 29,  8,  5, 67, 79,  1,  5, 99, 43, 20])
1466 torch.Size([32, 3, 32, 32]) tensor([43, 97, 86, 73, 20, 10, 48, 67, 37, 17, 42, 71, 39, 25, 66, 77, 95, 46,
        98, 34, 49, 25, 33, 38, 43, 96, 16, 22, 64, 95, 18, 85])
1467 torch.Size([32, 3, 32, 32]) tensor([75, 24, 81, 28, 71, 53, 84, 88, 88, 67, 69, 57, 95, 46, 16, 90, 20, 70,
        36, 76, 30, 87, 69, 77, 18, 68, 63, 86, 88, 67, 45,  3])
1468 torch.Size([32, 3, 32, 32]) tensor([11,  6, 51, 33, 76, 85, 31, 69, 73, 14, 11, 26, 55, 41,  0, 65, 47, 7

1518 torch.Size([32, 3, 32, 32]) tensor([31, 88,  0, 71, 83, 42,  7, 10, 38, 60, 83, 21, 55, 22, 24, 19,  1, 74,
        32, 64, 41, 35, 72, 27, 59, 18, 27, 18, 21, 84, 52, 52])
1519 torch.Size([32, 3, 32, 32]) tensor([56, 60, 57, 40, 95,  4, 36, 85, 62, 48,  2, 51, 53, 16, 13, 64, 12, 93,
        88, 36, 36, 98, 13, 74, 68, 76, 92,  0, 98, 48, 64, 52])
1520 torch.Size([32, 3, 32, 32]) tensor([19, 16, 86,  3, 56, 48, 43, 23, 26, 87, 85, 71, 85, 27, 98, 92, 39, 21,
        31, 16, 50, 60, 59, 43, 33, 40, 79, 94, 29, 52, 90, 50])
1521 torch.Size([32, 3, 32, 32]) tensor([81, 75, 35, 32, 83, 81,  4, 34, 29, 56, 77, 59, 56, 59, 12, 56, 24, 93,
        10,  5, 56, 67, 16, 14, 57, 55, 91, 46, 44, 12, 23, 58])
1522 torch.Size([32, 3, 32, 32]) tensor([80, 23, 49, 51, 95, 79, 87, 16, 41, 61, 69, 99,  6, 95, 87, 17, 95, 25,
        31, 29, 64, 16, 93, 47, 23,  3, 51, 89, 25, 45, 11, 24])
1523 torch.Size([32, 3, 32, 32]) tensor([78, 36, 17, 78, 70, 22, 52, 13, 10, 79,  1, 89, 39, 37, 73, 33, 35, 8

In [10]:
record_data = torch.zeros(16)
record = []
record_ones = torch.zeros(16)
for idx, (data, target) in enumerate(test_loader):
    record.append(target)
    record_data += target
    record_ones += torch.ones_like(target)
    print(idx, data.shape, target.shape, target)
    
record_data, record_ones

0 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([4, 2, 2, 0, 1, 7, 6, 1, 4, 7, 5, 8, 1, 3, 1, 4])
1 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([5, 0, 8, 0, 8, 2, 3, 1, 1, 8, 5, 5, 6, 4, 8, 2])
2 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([3, 7, 7, 0, 3, 9, 4, 2, 6, 7, 4, 2, 5, 2, 7, 5])
3 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([0, 6, 7, 8, 0, 6, 1, 8, 7, 2, 7, 6, 2, 0, 7, 6])
4 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([4, 7, 7, 0, 3, 2, 8, 6, 2, 2, 0, 3, 5, 3, 5, 2])
5 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([2, 7, 8, 3, 1, 4, 7, 4, 4, 7, 9, 7, 3, 3, 0, 9])
6 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([4, 7, 9, 3, 2, 3, 7, 2, 4, 2, 3, 8, 0, 1, 9, 8])
7 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([5, 4, 1, 0, 9, 3, 8, 1, 2, 7, 6, 8, 9, 8, 6, 5])
8 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([5, 6, 6, 1, 9, 7, 2, 1, 1, 6, 4, 9, 7, 2, 8, 3])
9 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([5, 2, 3, 

110 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([9, 6, 1, 3, 3, 9, 5, 1, 9, 1, 2, 6, 8, 3, 9, 6])
111 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([0, 3, 8, 4, 3, 2, 1, 2, 2, 2, 8, 1, 5, 3, 3, 9])
112 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([9, 5, 4, 3, 6, 6, 3, 7, 8, 6, 6, 2, 8, 8, 5, 9])
113 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([5, 8, 0, 4, 0, 6, 9, 6, 9, 2, 9, 0, 2, 7, 0, 2])
114 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([8, 7, 4, 2, 8, 7, 9, 7, 5, 9, 6, 9, 9, 5, 0, 2])
115 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([4, 3, 1, 9, 5, 4, 1, 9, 4, 5, 7, 9, 6, 7, 1, 1])
116 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([7, 3, 9, 9, 2, 4, 9, 2, 4, 8, 8, 8, 3, 8, 5, 2])
117 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([1, 7, 1, 0, 1, 3, 2, 5, 9, 7, 4, 2, 2, 2, 4, 9])
118 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([8, 9, 1, 9, 0, 8, 8, 4, 0, 0, 7, 1, 2, 4, 3, 3])
119 torch.Size([16, 3, 32, 32]) torch.Size([16

210 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([3, 0, 2, 2, 7, 5, 1, 0, 8, 8, 6, 9, 1, 7, 1, 6])
211 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([0, 1, 8, 9, 9, 8, 6, 5, 4, 8, 9, 1, 7, 6, 1, 9])
212 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([9, 6, 3, 8, 8, 3, 0, 6, 2, 0, 2, 2, 7, 8, 9, 8])
213 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([8, 1, 1, 5, 2, 4, 7, 7, 3, 4, 2, 8, 5, 3, 1, 6])
214 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([0, 1, 8, 6, 9, 8, 9, 2, 8, 4, 7, 5, 5, 0, 3, 9])
215 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([3, 5, 1, 5, 9, 5, 9, 0, 8, 2, 0, 1, 2, 8, 9, 9])
216 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([6, 7, 3, 5, 7, 3, 2, 0, 3, 4, 9, 6, 2, 1, 9, 0])
217 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([7, 4, 6, 5, 7, 4, 7, 1, 0, 5, 5, 1, 0, 2, 3, 6])
218 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([6, 6, 7, 5, 0, 2, 4, 8, 1, 6, 0, 7, 7, 4, 2, 5])
219 torch.Size([16, 3, 32, 32]) torch.Size([16

311 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([3, 6, 1, 0, 5, 5, 8, 5, 4, 4, 5, 5, 3, 2, 4, 5])
312 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([7, 7, 7, 7, 3, 3, 0, 2, 2, 7, 3, 2, 4, 4, 3, 0])
313 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([9, 3, 3, 4, 8, 8, 5, 7, 0, 7, 2, 9, 4, 1, 9, 8])
314 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([0, 2, 5, 3, 2, 6, 9, 3, 9, 2, 0, 3, 9, 1, 8, 1])
315 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([3, 6, 1, 7, 3, 7, 9, 3, 4, 6, 1, 3, 3, 0, 8, 1])
316 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([5, 0, 4, 9, 1, 6, 1, 8, 1, 3, 8, 7, 1, 5, 2, 1])
317 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([5, 5, 3, 3, 6, 4, 5, 3, 0, 0, 1, 1, 8, 1, 0, 2])
318 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([8, 9, 6, 6, 5, 2, 9, 8, 2, 6, 5, 5, 6, 6, 7, 6])
319 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([2, 8, 3, 3, 1, 7, 6, 3, 9, 8, 5, 1, 5, 6, 5, 6])
320 torch.Size([16, 3, 32, 32]) torch.Size([16

415 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([8, 0, 0, 2, 5, 6, 5, 9, 6, 4, 3, 4, 2, 6, 5, 2])
416 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([6, 2, 5, 3, 6, 8, 9, 8, 7, 4, 7, 5, 2, 2, 7, 8])
417 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([6, 2, 0, 1, 0, 3, 1, 9, 1, 6, 2, 6, 2, 8, 2, 7])
418 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([1, 5, 1, 9, 7, 9, 0, 3, 7, 4, 2, 1, 4, 6, 6, 0])
419 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([2, 8, 0, 9, 8, 4, 1, 5, 6, 6, 4, 5, 6, 6, 8, 4])
420 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([3, 9, 3, 8, 5, 5, 9, 5, 0, 7, 4, 5, 1, 4, 6, 3])
421 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([1, 5, 2, 2, 3, 2, 4, 1, 6, 2, 6, 8, 9, 2, 2, 2])
422 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([7, 8, 1, 9, 3, 5, 7, 6, 7, 7, 5, 8, 2, 7, 2, 9])
423 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([7, 6, 9, 8, 3, 1, 4, 8, 6, 1, 7, 1, 5, 3, 5, 8])
424 torch.Size([16, 3, 32, 32]) torch.Size([16

518 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([5, 6, 8, 1, 4, 8, 5, 5, 1, 3, 4, 0, 5, 4, 8, 3])
519 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([2, 8, 7, 1, 7, 3, 9, 8, 7, 6, 8, 8, 6, 7, 1, 1])
520 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([1, 1, 7, 7, 3, 0, 8, 8, 8, 6, 1, 5, 8, 8, 7, 3])
521 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([1, 1, 2, 3, 7, 8, 8, 3, 9, 0, 6, 1, 1, 6, 0, 2])
522 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([3, 9, 7, 7, 3, 8, 2, 2, 9, 1, 7, 3, 0, 8, 8, 4])
523 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([6, 6, 1, 3, 0, 2, 9, 8, 3, 4, 0, 4, 8, 1, 1, 7])
524 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([5, 1, 6, 3, 1, 1, 5, 9, 8, 2, 5, 3, 3, 9, 3, 2])
525 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([4, 7, 4, 3, 3, 1, 1, 3, 7, 4, 5, 7, 6, 4, 7, 8])
526 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([4, 1, 0, 9, 6, 7, 6, 4, 8, 1, 8, 0, 9, 2, 2, 3])
527 torch.Size([16, 3, 32, 32]) torch.Size([16

620 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([8, 3, 0, 5, 8, 1, 8, 8, 1, 6, 0, 8, 1, 7, 5, 9])
621 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([1, 0, 0, 2, 5, 2, 0, 5, 1, 1, 8, 0, 2, 5, 0, 8])
622 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([4, 9, 2, 9, 6, 4, 0, 4, 0, 3, 5, 0, 1, 8, 5, 8])
623 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([5, 4, 2, 4, 1, 3, 3, 1, 6, 1, 5, 8, 4, 3, 2, 8])
624 torch.Size([16, 3, 32, 32]) torch.Size([16]) tensor([1, 5, 3, 1, 2, 3, 1, 8, 3, 4, 3, 4, 0, 0, 6, 8])


(tensor([2985., 2769., 2730., 2883., 2748., 2889., 2751., 2788., 2787., 2828.,
         2874., 2830., 2797., 2773., 2772., 2796.]),
 tensor([625., 625., 625., 625., 625., 625., 625., 625., 625., 625., 625., 625.,
         625., 625., 625., 625.]))

In [8]:
record_data /= idx + 1
record_data

tensor([4.4864, 4.4848, 4.5344, 4.4976, 4.4832, 4.1568, 4.4112, 4.5248, 4.7024,
        4.5536, 4.5408, 4.3840, 4.6592, 4.5280, 4.3808, 4.6720])